# Sports Analytics - Reading Excel Files

In [12]:
# Import libraries needed to read and work with data
import pandas as pd
import numpy as np
import os

In [13]:
# Load the player data from a CSV file and show a preview
player_data = pd.read_csv('Data/player_data.csv')
print("Player Data:")
print(player_data.head())  # Show first few rows
print(f"\nShape: {player_data.shape}")  # Show how many rows and columns
print(f"Columns: {player_data.columns.tolist()}")  # Show column names

Player Data:
   player  rating
0  USA_p1     3.0
1  USA_p2     3.0
2  USA_p3     3.5
3  USA_p4     0.0
4  USA_p5     2.0

Shape: (144, 2)
Columns: ['player', 'rating']


In [17]:
# Load the game stint data from a CSV file and show a preview
stint_data = pd.read_csv('Data/stint_data.csv')
print("Stint Data:")
print(stint_data.head())  # Show first few rows
print(f"\nShape: {stint_data.shape}")  # Show how many rows and columns
print(f"Columns: {stint_data.columns.tolist()}")  # Show column names

Stint Data:
   game_id h_team a_team   minutes  h_goals  a_goals   home1   home2   home3  \
0        1    USA  Japan  4.252969        4        9  USA_p4  USA_p1  USA_p3   
1        1    USA  Japan  5.688809        6       11  USA_p1  USA_p7  USA_p6   
2        1    USA  Japan  1.149557        0        1  USA_p8  USA_p1  USA_p4   
3        1    USA  Japan  3.511617        7        5  USA_p2  USA_p7  USA_p8   
4        1    USA  Japan  2.163139        7        5  USA_p9  USA_p5  USA_p8   

    home4      away1      away2      away3      away4  
0  USA_p6  Japan_p12   Japan_p4   Japan_p8   Japan_p6  
1  USA_p5   Japan_p7  Japan_p10   Japan_p8  Japan_p12  
2  USA_p5   Japan_p5   Japan_p3   Japan_p7  Japan_p10  
3  USA_p6   Japan_p2   Japan_p4  Japan_p10  Japan_p12  
4  USA_p7   Japan_p8   Japan_p6  Japan_p10   Japan_p1  

Shape: (7448, 14)
Columns: ['game_id', 'h_team', 'a_team', 'minutes', 'h_goals', 'a_goals', 'home1', 'home2', 'home3', 'home4', 'away1', 'away2', 'away3', 'away4']


In [18]:
# Get final score per game
final_scores = (
    stint_data
    .groupby('game_id', as_index=False)
    .agg({
        'h_team': 'first',
        'a_team': 'first',
        'h_goals': 'max',
        'a_goals': 'max'
    })
)

# Determine winner
final_scores['winner'] = np.where(
    final_scores['h_goals'] > final_scores['a_goals'],
    final_scores['h_team'],
    np.where(
        final_scores['a_goals'] > final_scores['h_goals'],
        final_scores['a_team'],
        'Draw'
    )
)

final_scores

,game_id,h_team,a_team,h_goals,a_goals,winner
0,1,USA,Japan,8,13,Japan
1,2,USA,Great.Britain,10,7,USA
2,3,USA,Canada,13,6,USA
3,4,USA,France,12,8,USA
4,5,USA,Denmark,11,15,Denmark
...,...,...,...,...,...,...
655,656,Chile,Sweden,17,9,Chile
656,657,Chile,Brazil,10,12,Brazil
657,658,Chile,Columbia,15,9,Chile
658,659,Chile,Poland,10,9,Chile


In [20]:
player_cols_home = ['home1','home2','home3','home4']
player_cols_away = ['away1','away2','away3','away4']

def explode_stints(stints):
    rows = []
    for _, r in stints.iterrows():
        for side, cols in [('home', player_cols_home), ('away', player_cols_away)]:
            for p in cols:
                rows.append({
                    'game_id': r.game_id,
                    'player': r[p],
                    'side': side,
                    'minutes': r.minutes,
                    'team_goals': r.h_goals if side == 'home' else r.a_goals,
                    'opp_goals': r.a_goals if side == 'home' else r.h_goals,
                    'home_win': int(
                        (r.h_goals > r.a_goals) if side == 'home' else (r.a_goals > r.h_goals)
                    )
                })
    return pd.DataFrame(rows)

player_stints = explode_stints(stint_data)
player_stints

,game_id,player,side,minutes,team_goals,opp_goals,home_win
0,1,USA_p4,home,4.252969,4,9,0
1,1,USA_p1,home,4.252969,4,9,0
2,1,USA_p3,home,4.252969,4,9,0
3,1,USA_p6,home,4.252969,4,9,0
4,1,Japan_p12,away,4.252969,9,4,1
...,...,...,...,...,...,...,...
59579,660,Chile_p4,home,2.200405,5,2,1
59580,660,Argentina_p6,away,2.200405,2,5,0
59581,660,Argentina_p5,away,2.200405,2,5,0
59582,660,Argentina_p12,away,2.200405,2,5,0


In [21]:
player_stints['goal_diff_per_min'] = (
    (player_stints.team_goals - player_stints.opp_goals)
    / player_stints.minutes
)

player_stints['team_goals_per_min'] = player_stints.team_goals / player_stints.minutes
player_stints['opp_goals_per_min']  = player_stints.opp_goals  / player_stints.minutes
player_stints

,game_id,player,side,minutes,team_goals,opp_goals,home_win,goal_diff_per_min,team_goals_per_min,opp_goals_per_min
0,1,USA_p4,home,4.252969,4,9,0,-1.175649,0.940519,2.116169
1,1,USA_p1,home,4.252969,4,9,0,-1.175649,0.940519,2.116169
2,1,USA_p3,home,4.252969,4,9,0,-1.175649,0.940519,2.116169
3,1,USA_p6,home,4.252969,4,9,0,-1.175649,0.940519,2.116169
4,1,Japan_p12,away,4.252969,9,4,1,1.175649,2.116169,0.940519
...,...,...,...,...,...,...,...,...,...,...
59579,660,Chile_p4,home,2.200405,5,2,1,1.363385,2.272309,0.908923
59580,660,Argentina_p6,away,2.200405,2,5,0,-1.363385,0.908923,2.272309
59581,660,Argentina_p5,away,2.200405,2,5,0,-1.363385,0.908923,2.272309
59582,660,Argentina_p12,away,2.200405,2,5,0,-1.363385,0.908923,2.272309


In [23]:
ratings = player_data.set_index('player')['rating']

def add_teammate_features(df):
    df = df.copy()
    for gid in df.game_id.unique():
        mask = df.game_id == gid
        players = df.loc[mask, 'player']
        r = ratings.loc[players].values
        df.loc[mask, 'avg_teammate_rating'] = np.mean(r)
        df.loc[mask, 'sum_teammate_rating'] = np.sum(r)
        df.loc[mask, 'max_teammate_rating'] = np.max(r)
    return df

player_stints = add_teammate_features(player_stints)
player_stints

,game_id,player,side,minutes,team_goals,opp_goals,home_win,goal_diff_per_min,team_goals_per_min,opp_goals_per_min,avg_teammate_rating,sum_teammate_rating,max_teammate_rating
0,1,USA_p4,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5
1,1,USA_p1,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5
2,1,USA_p3,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5
3,1,USA_p6,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5
4,1,Japan_p12,away,4.252969,9,4,1,1.175649,2.116169,0.940519,1.875000,165.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59579,660,Chile_p4,home,2.200405,5,2,1,1.363385,2.272309,0.908923,1.920455,169.0,3.5
59580,660,Argentina_p6,away,2.200405,2,5,0,-1.363385,0.908923,2.272309,1.920455,169.0,3.5
59581,660,Argentina_p5,away,2.200405,2,5,0,-1.363385,0.908923,2.272309,1.920455,169.0,3.5
59582,660,Argentina_p12,away,2.200405,2,5,0,-1.363385,0.908923,2.272309,1.920455,169.0,3.5


In [24]:
opp_avg_diff = (
    player_stints.groupby('game_id')['goal_diff_per_min']
    .mean()
    .rename('avg_opp_diff')
)

player_stints = player_stints.merge(opp_avg_diff, on='game_id')
player_stints

,game_id,player,side,minutes,team_goals,opp_goals,home_win,goal_diff_per_min,team_goals_per_min,opp_goals_per_min,avg_teammate_rating,sum_teammate_rating,max_teammate_rating,avg_opp_diff
0,1,USA_p4,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5,0.0
1,1,USA_p1,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5,0.0
2,1,USA_p3,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5,0.0
3,1,USA_p6,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5,0.0
4,1,Japan_p12,away,4.252969,9,4,1,1.175649,2.116169,0.940519,1.875000,165.0,3.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59579,660,Chile_p4,home,2.200405,5,2,1,1.363385,2.272309,0.908923,1.920455,169.0,3.5,0.0
59580,660,Argentina_p6,away,2.200405,2,5,0,-1.363385,0.908923,2.272309,1.920455,169.0,3.5,0.0
59581,660,Argentina_p5,away,2.200405,2,5,0,-1.363385,0.908923,2.272309,1.920455,169.0,3.5,0.0
59582,660,Argentina_p12,away,2.200405,2,5,0,-1.363385,0.908923,2.272309,1.920455,169.0,3.5,0.0


In [25]:
player_stints['win_weighted_diff'] = (
    player_stints.goal_diff_per_min * (1 + player_stints.home_win)
)
player_stints

,game_id,player,side,minutes,team_goals,opp_goals,home_win,goal_diff_per_min,team_goals_per_min,opp_goals_per_min,avg_teammate_rating,sum_teammate_rating,max_teammate_rating,avg_opp_diff,win_weighted_diff
0,1,USA_p4,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5,0.0,-1.175649
1,1,USA_p1,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5,0.0,-1.175649
2,1,USA_p3,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5,0.0,-1.175649
3,1,USA_p6,home,4.252969,4,9,0,-1.175649,0.940519,2.116169,1.875000,165.0,3.5,0.0,-1.175649
4,1,Japan_p12,away,4.252969,9,4,1,1.175649,2.116169,0.940519,1.875000,165.0,3.5,0.0,2.351299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59579,660,Chile_p4,home,2.200405,5,2,1,1.363385,2.272309,0.908923,1.920455,169.0,3.5,0.0,2.726770
59580,660,Argentina_p6,away,2.200405,2,5,0,-1.363385,0.908923,2.272309,1.920455,169.0,3.5,0.0,-1.363385
59581,660,Argentina_p5,away,2.200405,2,5,0,-1.363385,0.908923,2.272309,1.920455,169.0,3.5,0.0,-1.363385
59582,660,Argentina_p12,away,2.200405,2,5,0,-1.363385,0.908923,2.272309,1.920455,169.0,3.5,0.0,-1.363385


In [26]:
features = [
    'team_goals_per_min',
    'opp_goals_per_min',
    'avg_teammate_rating',
    'sum_teammate_rating',
    'max_teammate_rating',
    'avg_opp_diff',
    'minutes',
    'home_win'
]

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [28]:
X = player_stints[features]
y = player_stints['goal_diff_per_min']

In [29]:
param_grid = {
    'n_estimators': [200, 400, 600],
    'max_depth': [6, 10, 14, None],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

grid = GridSearchCV(
    rf,
    param_grid,
    scoring='r2',
    cv=5
)

grid.fit(X, y)
best_rf = grid.best_estimator_


KeyboardInterrupt: 